This script works using the lbry daemon so you must have lbrynet
running to execute this script. Currently a work in progress— will
update following successful history construction and also upload
history .csv

In [ ]:
from jsonrpc.proxy import JSONRPCProxy
daemon = JSONRPCProxy.from_url("http://localhost:5279/lbryapi")

In [ ]:
daemon.get_balance() #this is just a test

In [ ]:
import json
import requests
import datetime

In [ ]:
#jacks reward function
COIN = 10**8
block_time = 150
blocks_per_ramp = 100
start_ramp_height = 5001
limit = 1083221000 * COIN
subsidy_interval = 1 << 5
max_reductions = 500
peak_reward = 500 * COIN
premine = 400000000 * COIN


def reward(n):
    assert n >= 0
    if n < 5001:
        return COIN
    elif n < 55001:
        r = 1 + ((n - ((n - start_ramp_height) % blocks_per_ramp) - start_ramp_height) / blocks_per_ramp)
        return r * COIN
    else:
        level = int((n - 55001) / subsidy_interval)
        reduction = int((-1 + int((8 * level) + 1) ** 0.5) / 2)
        while level >= ((reduction * reduction + reduction) >> 1):
            if ((reduction * reduction + reduction) >> 1) > level:
                reduction -= 1
            else:
                reduction += 1
        if reduction * COIN >= peak_reward:
            return 0
        return peak_reward - (reduction * COIN)

In [ ]:
def block_velocity(block):
    list_of_tx = [daemon.get_transaction({'txid':txh}) for txh in block['tx']]
    #print list_of_tx
    list_of_tx_outs = [sum([int(out['value'])for out in tx['outputs']]) for tx in list_of_tx]
    return float(sum(list_of_tx_outs))/COIN

In [ ]:
conv = COIN #deweys to 1 LBC
block_data = {}

#genesis
genesis_hash = "9c89283ba0f3227f6c03b70216b9f665f0118d5e0fa729cedf4fb34d6a34f463"

current_block=daemon.get_block({'blockhash': genesis_hash})

block_data[0] = {"height":0, 
                     "timestamp":datetime.datetime.fromtimestamp(int(current_block['time'])), 
                     "reward":reward(0)/COIN, 
                     "tx_count": len(current_block['tx']),
                     "tx_outs": block_velocity(current_block),
                     "difficulty": current_block['difficulty'],
                     "confirmations": current_block['confirmations']
                 }

In [ ]:
block_data

In [ ]:
bhash=daemon.get_best_blockhash()
current_block=daemon.get_block({'blockhash': bhash})

In [ ]:
block_count = current_block['height']
current_block

In [ ]:
vel_fail = []
for j in range(block_count,0,-1):
    try:
        velocity = block_velocity(current_block)
    except:
        velocity = None
        print "block "+str(j)+" velocity computation failed at "+str(datetime.datetime.now())
        vel_fail = vel_fail+[j]
        
    block_data[j] = {"height":j, 
                     "timestamp":datetime.datetime.fromtimestamp(int(current_block['time'])), 
                     "reward":reward(j)/COIN, 
                     "tx_count": len(current_block['tx']),
                     "tx_outs": velocity,
                     "difficulty": current_block['difficulty'],
                     "confirmations": current_block['confirmations']
                    }
    del velocity
    previous_blockhash = current_block['previousblockhash']
    current_block=daemon.get_block({'blockhash': previous_blockhash})


In [ ]:
df = pd.DataFrame(block_data).T.order_by("height")

In [ ]:
df.head()

In [ ]:
df['mined_supply'] = df[df.height>0].reward.cumsum()
df['total_supply'] = df.mined_supply+premine

In [ ]:
df.to_csv('lbry_history.csv')